## 对指定的 state_of_the_union.txt 进行检索

In [35]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader

# 文档加载
loader = TextLoader('./三国演义第1回.txt', encoding='utf8')
loader

In [36]:
from langchain.indexes import VectorstoreIndexCreator

# 创建向量索引
index = VectorstoreIndexCreator().from_loaders([loader])
index

VectorStoreIndexWrapper(vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x00000236FCDF98D0>)

In [37]:
# 检索文档中的内容
query = "桃园三结义是哪三个人"
#query = "水浒传"
index.query(query)

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


' 刘备、关羽、张飞'

In [38]:
index.query_with_sources(query)

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


{'question': '桃园三结义是哪三个人',
 'answer': ' 刘备、关羽、张飞。\n',
 'sources': './三国演义前1回.txt'}

## Walkthrough

In [39]:
documents = loader.load()
documents

[Document(page_content='三国演义\n作者：罗贯中\n\n正文 第一回 宴桃园豪杰三结义 斩黄巾英雄首立功\n\n滚滚长江东逝水，浪花淘尽英雄。是非成败转头空。\n\u3000\u3000青山依旧在，几度夕阳红。\u3000\u3000白发渔樵江渚上，惯\n\u3000\u3000看秋月春风。一壶浊酒喜相逢。古今多少事，都付\n\u3000\u3000笑谈中。\n\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000——调寄《临江仙》\n话说天下大势，分久必合，合久必分。周末七国分争，并入于秦。及秦灭之后，楚、汉分争，又并入于汉。汉朝自高祖斩白蛇而起义，一统天下，后来光武中兴，传至献帝，遂分为三国。推其致乱之由，殆始于桓、灵二帝。桓帝禁锢善类，崇信宦官。及桓帝崩，灵帝即位，大将军窦武、太傅陈蕃共相辅佐。时有宦官曹节等弄权，窦武、陈蕃谋诛之，机事不密，反为所害，中涓自此愈横。\n建宁二年四月望日，帝御温德殿。方升座，殿角狂风骤起。只见一条大青蛇，从梁上飞将下来，蟠于椅上。帝惊倒，左右急救入宫，百官俱奔避。须臾，蛇不见了。\n次日，于桃园中，备下乌牛白马祭礼等项，三人焚香再拜而说誓曰：“念刘备、关羽、张飞，虽然异姓，既结为兄弟，则同心协力，救困扶危；上报国家，下安黎庶。不求同年同月同日生，只愿同年同月同日死。皇天后土，实鉴此心，背义忘恩，天人共戮！”誓毕，拜玄德为兄，关羽次之，张飞为弟。祭罢天地，复宰牛设酒，聚乡中勇士，得三百余人，就桃园中痛饮一醉。来日收拾军器，但恨无马匹可乘。正思虑间，人报有两个客人，引一伙伴当，赶一群马，投庄上来。玄德曰：“此天佑我也！”三人出庄迎接。原来二客乃中山大商：一名张世平，一名苏双，每年往北贩马，近因寇发而回。玄德请二人到庄，置酒管待，诉说欲讨贼安民之意。二客大喜，愿将良马五十匹相送；又赠金银五百两，镔铁一千斤，以资器用。', metadata={'source': './三国演义前1回.txt'})]

In [43]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
len(texts)
#texts[0]

1

In [41]:
from langchain.embeddings import OpenAIEmbeddings
# 使用OpenAI embedding
embeddings = OpenAIEmbeddings()

from langchain.vectorstores import Chroma
# 创建向量存储 db
db = Chroma.from_documents(texts, embeddings)
db

In [42]:
# 检索接口
retriever = db.as_retriever()

qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)
query = "桃园三结义都有谁？"
qa.run(query)

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


' 刘备、关羽、张飞。'